<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#In-this-notebook," data-toc-modified-id="In-this-notebook,-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>In this notebook,</a></span></li><li><span><a href="#Expected-utility-via-Monte-Carlo" data-toc-modified-id="Expected-utility-via-Monte-Carlo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Expected utility via Monte Carlo</a></span><ul class="toc-item"><li><span><a href="#Plotting-the-exact-solution" data-toc-modified-id="Plotting-the-exact-solution-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Plotting the exact solution</a></span></li><li><span><a href="#MC-implementation" data-toc-modified-id="MC-implementation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>MC implementation</a></span></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exercise</a></span></li></ul></div>

# 4. Expected utility and Monte Carlo in Python
**Camilo A. Garcia Trillos - 2020**

---

## In this notebook,
- we look at the Monte Carlo method and how to use it to approximate expected utilities or certainty equivalents.
- we use Python to plot information using matplotlib, including a histogram and a regression
---

Let us import some packages: math, numpy, matplotlib and scipy

In [ ]:
import math
import numpy as np
import scipy as sp
from numpy.random import default_rng  #  pseudo-random number generator
import matplotlib.pyplot as plt

# This is an indicator to tell jupyter notebook to show us all plots inline:
%matplotlib inline 


## Expected utility via Monte Carlo

To compute the expected utility of a wealth gamble $W$ we can use he law of large numbers. Indeed, if $E[|u(W)|]<\infty$, we have
$$  \frac{1}{N} \sum_{i=1}^N u(W_i)  \rightarrow \mathbb E[u(W)] \text{ as } N\rightarrow \infty,$$
where $(W_i)$ is a family of independent draws of random variables with $W_i \sim W$ for each $i$.

The Monte Carlo method relies on this equality to produce an approximation to the expectation (by choosing a large N and calculating the empirical average).


To see how this works, let us start with $W$ being normally distributed, that is $W = \sigma N + \mu$, where $\mu, \sigma \in \mathbb R$ and $N$ is standard normally distributed.  

Now, let us suppose first we want to compute expected utility of a CARA utility $u(x) = 1-\exp(-\alpha * x)$. We can calculate explicitly
$$ \mathbb E[u(W)] = \mathbb E[1- \exp(-\alpha \sigma N - \alpha \mu  ))] =1- \exp\left(-\alpha \mu + \frac 1 2 \alpha^2 \sigma^2  \right).$$

We use this value to compare to the value approximated by Monte Carlo as explained before. Let us build a plot of this function in some given domain.

### Plotting the exact solution

There are several libraries allowing us to plot in Python. We will use one of the simplest: Matplotlib.

A simple way to plot in this library is to provide it with vectors of input and output. To try it, let us simply plot the result of the (exact) expected utility when the CARA coefficient changes.

We start by sampling the space of coefficients of risk aversion:

In [ ]:
x = np.linspace(0.001,3,100) # creates a vector of size 100 with numbers between 0.1 and 30
print(x)

We now implement the exact solution expected CARA utility under normal assumptions. Since it is a simple expression, we can use a lambda function as introduced before.

In [ ]:
# The operations in expected_u are well defined for vectors as long as mu,sd,x broadcast correctly together.
expected_u = lambda mu,sigma,alpha: 1-np.exp(-alpha*mu+0.5*alpha**2*sigma**2)

Note that we use 'np.exp' and not 'math.exp': this is because we want the function to be 'vectorial', that is, to accept vectors as an input 

(try changing np.exp for math.exp, run the code and then run the code below... there will be an error).

In [ ]:
sd, mu = 2,5  # Equivalently sd=2 and mu=5
y=expected_u(mu,sd,x) # Note that x is a vector
print(y) # And so is y

If for some reason you cannot implement directly a vectorial function, it is possible to use a loop or the function np.vectorize to render the function vector ready.

We are ready to make the plot:

In [ ]:
plt.plot(x,y)   # Make a plot between x and y
plt.title('Expected utility as a function of coefficient of absolute risk aversion - Gaussian case') # Add a title
plt.xlabel('Coefficient of risk aversion') # Add a label on the x axis
plt.ylabel('Expected utility') # Add a label on the y axis

### MC implementation

Let us now look at the Monte Carlo approximation of the above function. We start by defining a function that calculates the CARA utility:

In [ ]:
cara_utility = lambda x,alpha: 1-np.exp(-alpha*x)

In [ ]:
# Some tests on our function
assert cara_utility(1,1)== 1-1./math.e , "Failed test with x=1, alpha =1"
assert cara_utility(5,2)== 1-math.e**-10., "Failed test with x=5, alpha=2"

We can now generate a sample of wealths, distributed like a $\mathcal N (\mu,\sigma^2)$. 

In [ ]:
sd, mu = 2,5  # Equivalently sd=2 and mu=5
N = 10000
rng = default_rng()
sample_gaussian = rng.normal(mu,sd,N)

How can we check that these are Gaussian? We can plot the histogram of the empirical distribution defined. The package matplotlib has a convenient function for this: *plt.hist* (recall that plt is our alias for pyplot)

In [ ]:
plt.hist(sample_gaussian, density=True)  # Plots the histogram, normalising to obtain a pdf.
plt.title('Histogram of our sample') # Add a title to the plot
plt.xlabel('sample_gaussian') # Add a label on the X axis
plt.ylabel('density') # Add a label on the Y axis

It looks like a good Gaussian sample with our parameters (centred in 5 and with standard deviation 2). In later notebooks we will learn some alternative ways for checking Gaussianity. 

We can now calculate a Monte Carlo approximation of our expected utility. Examine the code below:

In [ ]:
cara_utility(sample_gaussian,1).mean()  # In one line, we evaluate the cara utility for each entry of the sample, and then calculate the mean of the resulting vector

Observe now the following: the estimation is random. To see this, let us run the estimation with another sample

In [ ]:
cara_utility(rng.normal(mu,sd,N),1).mean()  

As expected, the two values are close but different. Indeed, this estimator is **random**, because it depends on the sample, which is itself random. This is something to be taken into account when using Monte Carlo estimators. 

In fairness, the Python implementation of the MC estimator can only produce a pseudo-random generation. To see this, we can fix the seed of the pseudo-random generation algorithm and compare the answers

In [ ]:
rng = default_rng(1234)
sample_gaussian = rng.normal(mu,sd,N)
mc_eu1 = cara_utility(sample_gaussian,1).mean()
rng = default_rng(1234)
sample_gaussian2 = rng.normal(mu,sd,N)
mc_eu2 = cara_utility(sample_gaussian2,1).mean()
print(mc_eu1-mc_eu2)

Setting the random states allows us to repeat the same sequence on the pseudo-random generation.

Now, let us remind ourselves of the closed form solution:

In [ ]:
expected_u(mu,sd,1)

We see that the value is very close to the value(s) estimated via MC. Indeed, this error can be explained via the central limit theorem, which give us a control on the L_2 norm and is of the form

$$\left \|\mathbb E[u(W)] - \frac 1 N \sum_{i=1}^N u(W_i)    \right \|_{L_2} \leq \frac{C}{N^{1/2}} {\rm sd}(X_1)  $$

Let us verify this empirically, using a plot and a regression. We want to retrieve the rate of convergence, which is the power 1/2 in the above control. To do this we use a log-log plot (think why).

In [ ]:
n_vec = 2**np.arange(10,20) # The number of MC simulations, we take powers of 2
rng = default_rng(1) # Fix the seed to "1", so that the plot looks the same every time you run it
u = np.array([cara_utility(rng.normal(mu,sd,N),1).mean() for N in n_vec])  # Create an MC expected utility for the sizes above
error = np.abs(u - expected_u(mu, sd, 1)) # Calculate the error
plt.loglog(n_vec, error, 'ro', label='Error') # Make a log log plot
plt.title('Error in Monte Carlo Expected utility as a function of size of sample - Gaussian case')
plt.xlabel('N')
plt.ylabel('Error')

# Let us also add a reference line. To do so, we need to calculate a simple regression. We can use the polyfit function
m, b = np.polyfit( np.log(n_vec), np.log(error), 1)
plt.loglog(n_vec, np.exp(b+m*np.log(n_vec)), 'g--', label='Best fit: Error ='+ "%.2f N^(%.2f)" % (math.exp(b),m))   
plt.legend()

The hardest line of code in the plot above is possibly
```python
plt.loglog(n_vec, np.exp(b+m*np.log(n_vec)), 'g--', label='Best fit: Error ='+ "%.2f N^(%.2f)" % (math.exp(b),m))   
```

Let us look at two parts in particular:

```python
'g--'
```
Means make a green dashed line.

while 
```python
label='Best fit: Error ='+ "%.2f N^(%.2f)" % (math.exp(b),m))   
```
means: take the value of exp(b), round it to a float with two decimal figures, do the same with m, and write a string that contains exp(b) N^ m with this format. This is saved on a variable label that is used by matplotlib to assign the legends in a plot.

Check that you understand the other lines of code.

Note that the best fit slope is close to -1/2 as expected. This is consistent with the theoretical error given before. **Write the equations to be sure you understand why.**

## Exercise

1. Compute, via a Monte-Carlo simulation, the expected utility of a CRRA investor for the following gambles.
    - $W_1 \sim |aN + b|$, where $N$ is standard normally distributed and $a,b \in R$.
    - $W_2 \sim \text{Exp}(\lambda_2)$ where $\lambda_2>0$.
	
You might have to look up online the commands for the corresponding random number generators. (Use the ones in numpy.random).

2. Write a function that computes the certainty equivalent for a CRRA investor. (Hint: You might have to compute, on a piece of paper, $u^{-1}$ for the different relative risk aversions $\rho$.)

3. With $a = 1$ and $b = 2$, plot the certainty equivalent of a CRRA investor as a function of relative risk aversion $\rho$, using gamble $W_1$. 